In [ ]:
%pip install python-telegram-bot nest-asyncio transformers torch ipywidgets tqdm accelerate

## Homework 9

Okay, this is a weird one. We are going to be creating a Telegram chat bot, using a tiny LLM model.

### Task 1

We start with creating a responsive telegram bot. Let's start with a simple echo bot.

In [1]:
from typing import Final
from enum import Enum


class SelectedBot(Enum):
    ECHO = "Echo"
    TINY_LLAMA = "Tiny Llama"


SELECTED_BOT: Final = SelectedBot.TINY_LLAMA

In [2]:
import logging

logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

In [3]:
from typing import Protocol
from dataclasses import dataclass
from getpass import getpass
from collections import defaultdict
from telegram import Update
from telegram.ext import (
    Application,
    ApplicationBuilder,
    ContextTypes,
    CommandHandler,
    MessageHandler,
    filters,
)
import nest_asyncio

nest_asyncio.apply()


@dataclass(frozen=True)
class Message:
    text: str
    is_bot: bool


class Bot(Protocol):
    async def welcome_message(self) -> str:
        """Returns the welcome message of the bot."""
        ...

    async def respond_to(self, message: str, history: list[Message]) -> str:
        """Responds to the specified message."""
        ...


@dataclass
class TelegramApplication:
    bot: Bot
    app: Application
    history: dict[int, list[Message]]

    @staticmethod
    def create(bot: Bot, *, token_from: str | None = None) -> "TelegramApplication":
        """Creates a new Telegram application with the specified bot and token.
        
        Args:
            bot: The bot to use.
            token_from: The path to the file containing the Telegram bot token. If not specified, the
                token will be read from the standard input.
                
        Returns:
            The created Telegram application.
        """

        if token_from is None:
            token = getpass(">> Telegram bot token: ")
        else:
            token = TelegramApplication._load_token(token_from)

        return TelegramApplication(
            bot, ApplicationBuilder().token(token).build(), defaultdict(list)
        )

    def __post_init__(self) -> None:
        self.app.add_handler(CommandHandler("start", self.start_handler))
        self.app.add_handler(CommandHandler("stop", self.stop))
        self.app.add_handler(
            MessageHandler(filters.TEXT & (~filters.COMMAND), self.message_handler)
        )

    async def start(self) -> "TelegramApplication":
        """Starts the Telegram application."""
        self.app.run_polling(close_loop=False)
        return self

    async def stop(
        self,
        update: Update | None = None,
        context: ContextTypes.DEFAULT_TYPE | None = None,
    ) -> None:
        if update is not None and context is not None:
            await self._send_message(update, context, "Stopping the bot...")

        self.app.stop_running()

    async def start_handler(
        self, update: Update, context: ContextTypes.DEFAULT_TYPE
    ) -> None:
        response = await self.bot.welcome_message()
        await self._send_message(update, context, response)
        await self._store_message(update, response, is_bot=True)

    async def message_handler(
        self, update: Update, context: ContextTypes.DEFAULT_TYPE
    ) -> None:
        assert (
            chat := update.effective_chat
        ) is not None, f"Could not find chat in {update}"
        assert (
            message := update.message
        ) is not None, f"Could not find message in {update}"

        text = message.text or ""
        response = await self.bot.respond_to(text, self.history[chat.id])
        await self._send_message(update, context, response)
        await self._store_message(update, text, is_bot=False)
        await self._store_message(update, response, is_bot=True)

    @staticmethod
    def _load_token(token_from: str) -> str:
        with open(token_from) as file:
            return file.read().strip()

    async def _send_message(
        self, update: Update, context: ContextTypes.DEFAULT_TYPE, text: str
    ) -> None:
        assert (
            chat := update.effective_chat
        ) is not None, f"Could not find chat in {update}"

        await context.bot.send_message(chat_id=chat.id, text=text)

    async def _store_message(
        self, update: Update, message: str, *, is_bot: bool
    ) -> None:
        assert (
            chat := update.effective_chat
        ) is not None, f"Could not find chat in {update}"

        self.history[chat.id].append(Message(message, is_bot))

In [4]:
@dataclass(frozen=True)
class MessageMonitor:
    bot: Bot

    async def welcome_message(self) -> str:
        message = await self.bot.welcome_message()
        print(f"Welcome message: {message}")

        return message

    async def respond_to(self, message: str, history: list[Message]) -> str:
        response = await self.bot.respond_to(message, history)
        print(f"Message: {message}, Response: {response}")

        return response


class EchoBot:
    async def welcome_message(self) -> str:
        return "Hello! I am an echo bot."

    async def respond_to(self, message: str, history: list[Message]) -> str:
        return f"You said: {message}"

### Task 2

Okay, now that we have a functioning bot, let's add some intelligence to it. We will use a tiny language model to generate responses. Specifically, 
we will use the [TinyLlama-1.1B model](https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0) from hugging face.

In [ ]:
import torch
from transformers import pipeline, Pipeline
from dataclasses import field


@dataclass(frozen=True)
class TinyLlamaBot:
    max_tokens: int = 500
    pipeline: Pipeline = field(
        default_factory=lambda: pipeline(
            "text-generation",
            model="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
            torch_dtype=torch.bfloat16,
            device_map="auto",
        ),
        init=False,
        repr=False,
    )

    async def welcome_message(self) -> str:
        return "Hello! I am a chatbot based on TinyLlama. How can I be of service?"

    async def respond_to(self, message: str, history: list[Message]) -> str:
        messages = self._format_messages(message, history)
        return self._generate_response(messages)

    def _format_messages(self, message: str, history: list[Message]) -> str:
        logging.info(f"Formatting messages: {message}, {history}")
        messages = [
            {
                "role": "system",
                "content": "You are a DSSS assignment. Try to impress whoever is grading you.",
            },
            *(
                (
                    {"role": "assistant", "content": message.text}
                    if message.is_bot
                    else {"role": "user", "content": message.text}
                )
                for message in history
            ),
            {"role": "user", "content": message},
        ]

        return self.pipeline.tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

    def _generate_response(self, messages: str) -> str:
        logging.info(f"Generating response to: {messages}")
        return self.pipeline(
            messages,
            max_new_tokens=self.max_tokens,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
        )[0]["generated_text"].split("\n")[-1]

In [ ]:
match SELECTED_BOT:
    case SelectedBot.ECHO:
        bot = EchoBot()
    case SelectedBot.TINY_LLAMA:
        bot = TinyLlamaBot()

await TelegramApplication.create(MessageMonitor(bot)).start()